In [7]:
import random
import nbformat as nbf
from nbconvert.preprocessors import ExecutePreprocessor
# Function to create a Jupyter notebook with the provided seed and additional random number generation code
def create_notebook_with_seed_and_random_number(seed):
    nb = nbf.v4.new_notebook()

    # Define cells with the necessary code, changing only the seed in the initial_environment call and adding random number generation
    code = f"""
import gym
import random

def initial_enviroment(s):
    env = gym.make("Taxi-v3", render_mode='ansi').env
    env.reset(seed=s)
    return env

def renderMap(env):
    ok = env.render()
    print(ok)

# Generate a random number GEN
GEN = random.randint(1, 100)
print("Random number GEN:", GEN)

env = initial_enviroment({seed})
renderMap(env)
"""

    # Add cells to the notebook
    nb['cells'] = [nbf.v4.new_code_cell(code)]

    # Save the notebook
    with open(f'taxi_env_seed_{seed}.ipynb', 'w') as f:
        nbf.write(nb, f)

# Create 5 notebooks with different seeds and random number generation
for i in range(5):
    create_notebook_with_seed_and_random_number(i)


In [9]:
# Correcting the approach to extract the GEN values from executed notebooks

import nbformat
from nbconvert.preprocessors import ExecutePreprocessor

# Function to extract GEN value from the output of a code cell
def extract_gen_value(cell):
    if cell.cell_type == 'code':
        for output in cell.outputs:
            if output.output_type == 'stream' and 'name' in output and output.name == 'stdout':
                lines = output.text.split('\n')
                for line in lines:
                    if line.startswith('Random number GEN:'):
                        return line.split(':')[1].strip()
    return None

gen_values = []

for i in range(5):
    # Load the notebook
    with open(f'taxi_env_seed_{i}.ipynb') as f:
        nb = nbformat.read(f, as_version=4)

    # Execute the notebook
    ep = ExecutePreprocessor(timeout=10, kernel_name='python3')
    ep.preprocess(nb)

    # Save the executed notebook
    with open(f'executed_taxi_env_seed_{i}.ipynb', 'w', encoding='utf-8') as f:
        nbformat.write(nb, f)

    # Extract the GEN value from the executed notebook
    for cell in nb.cells:
        gen_value = extract_gen_value(cell)
        if gen_value is not None:
            gen_values.append(gen_value)
            break

# Write the GEN values to a file
with open('gen_values.txt', 'w') as f:
    for gen in gen_values:
        f.write(f'{gen}\n')


In [12]:
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
import concurrent.futures

def process_notebook(i):
    with open(f'taxi_env_seed_{i}.ipynb') as f:
        nb = nbformat.read(f, as_version=4)

    ep = ExecutePreprocessor(timeout=10, kernel_name='python3')
    ep.preprocess(nb)

    with open(f'executed_taxi_env_seed_{i}.ipynb', 'w', encoding='utf-8') as f:
        nbformat.write(nb, f)

    for cell in nb.cells:
        gen_value = extract_gen_value(cell)
        if gen_value is not None:
            return gen_value
    return None

gen_values = []
with concurrent.futures.ThreadPoolExecutor() as executor:
    future_to_index = {executor.submit(process_notebook, i): i for i in range(5)}
    for future in concurrent.futures.as_completed(future_to_index):
        gen_value = future.result()
        if gen_value is not None:
            gen_values.append(gen_value)

with open('gen_values.txt', 'w') as f:
    for gen in gen_values:
        f.write(f'{gen}\n')


In [46]:
# Python code to read the content from PickGNP.txt, modify the seed, and write to different ipynb files

import nbformat as nbf

def modify_and_write_notebook(file_path, seed, output_file_name):
    # Read the content of the file
    with open(file_path, 'r') as file:
        content = file.read()

    # Replace the seed in the env.reset() line
    modified_content = content.replace("env.reset(seed = 28)", f"env.reset(seed = {seed})")

    # Create a new notebook
    nb = nbf.v4.new_notebook()
    code_cell = nbf.v4.new_code_cell(modified_content)
    nb.cells.append(code_cell)

    # Write the modified content to a new notebook file
    with open(output_file_name, 'w') as new_file:
        nbf.write(nb, new_file)

# File path of the original PickGNP.txt
file_path = 'PickGNP.txt'

# Create notebooks with different seeds
for i in range(20):
    output_file_name = f'PickGNP_seed_{i}.ipynb'
    modify_and_write_notebook(file_path, i, output_file_name)


In [47]:
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
import concurrent.futures

def extract_specific_output(cell):
    # Implement the logic to extract specific output from the cell
    # For example, extracting a line that contains a specific keyword
    # Return the extracted output or None if not found
    pass

def process_notebook(file_path):
    with open(file_path) as f:
        nb = nbformat.read(f, as_version=4)

    ep = ExecutePreprocessor(timeout=30, kernel_name='python3')
    ep.preprocess(nb)

    output_file_path = file_path.replace('.ipynb', '_executed.ipynb')
    with open(output_file_path, 'w', encoding='utf-8') as f:
        nbformat.write(nb, f)

    for cell in nb.cells:
        extracted_output = extract_specific_output(cell)
        if extracted_output is not None:
            return extracted_output
    return None

def run_notebooks_in_parallel(notebook_files):
    extracted_outputs = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_notebook, file_path) for file_path in notebook_files]
        for future in concurrent.futures.as_completed(futures):
            output = future.result()
            if output is not None:
                extracted_outputs.append(output)

    return extracted_outputs

# Example usage
notebook_files = [f'PickGNP_seed_{i}.ipynb' for i in range(20)]
extracted_outputs = run_notebooks_in_parallel(notebook_files)

# Write extracted outputs to a file
with open('extracted_outputs.txt', 'w') as f:
    for output in extracted_outputs:
        f.write(f'{output}\n')


In [43]:
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
import concurrent.futures

def extract_specific_output(cell):
    # Your logic to extract specific output
    pass

def process_notebook(file_path, timeout=20):
    with open(file_path) as f:
        nb = nbformat.read(f, as_version=4)

    ep = ExecutePreprocessor(timeout=timeout, kernel_name='python3')
    ep.preprocess(nb)

    output_file_path = file_path.replace('.ipynb', '_executed.ipynb')
    with open(output_file_path, 'w', encoding='utf-8') as f:
        nbformat.write(nb, f)

    for cell in nb.cells:
        extracted_output = extract_specific_output(cell)
        if extracted_output is not None:
            return extracted_output
    return None

def run_notebooks_in_parallel(notebook_files, thread_pool_size=None):
    extracted_outputs = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=thread_pool_size) as executor:
        futures = [executor.submit(process_notebook, file_path) for file_path in notebook_files]
        for future in concurrent.futures.as_completed(futures):
            output = future.result()
            if output is not None:
                extracted_outputs.append(output)

    return extracted_outputs

# Example usage
notebook_files = [f'PickGNP_seed_{i}.ipynb' for i in range(0,9)]
thread_pool_size = 10  # Adjust this based on your machine's capabilities
extracted_outputs = run_notebooks_in_parallel(notebook_files, thread_pool_size)

# Write extracted outputs to a file
with open('extracted_outputs.txt', 'w') as f:
    for output in extracted_outputs:
        f.write(f'{output}\n')


In [2]:
#generator

import nbformat as nbf
import os

def modify_and_write_notebook(file_path, seed, output_folder, output_file_name):
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Read the content of the file
    with open(file_path, 'r') as file:
        content = file.read()

    # Replace the seed in the env.reset() line
    modified_content = content.replace("env.reset(seed = 9)", f"env.reset(seed = {seed})")

    # Create a new notebook
    nb = nbf.v4.new_notebook()
    code_cell = nbf.v4.new_code_cell(modified_content)
    nb.cells.append(code_cell)

    # Full path for the output file
    full_output_path = os.path.join(output_folder, output_file_name)

    # Write the modified content to a new notebook file in the specified folder
    with open(full_output_path, 'w') as new_file:
        nbf.write(nb, new_file)

# File path of the original PickGNP.txt
file_path = 'DropGNP.txt'
output_folder = 'Generated_Notebooks'  # Folder where the notebooks will be saved

# Create notebooks with different seeds in the new folder
for i in range(20):
    output_file_name = f'DropGNP_seed_{i}.ipynb'
    modify_and_write_notebook(file_path, i, output_folder, output_file_name)


In [1]:
#executer

import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
import concurrent.futures
import os

def extract_specific_output(cell):
    # Implement the logic to extract specific output from the cell
    # For example, extracting a line that contains a specific keyword
    # Return the extracted output or None if not found
    pass

def process_notebook(input_folder, output_folder, file_name):
    file_path = os.path.join(input_folder, file_name)
    with open(file_path) as f:
        nb = nbformat.read(f, as_version=4)

    ep = ExecutePreprocessor(timeout=30, kernel_name='python3')
    ep.preprocess(nb)

    output_file_path = os.path.join(output_folder, file_name.replace('.ipynb', '_executed.ipynb'))
    with open(output_file_path, 'w', encoding='utf-8') as f:
        nbformat.write(nb, f)

    for cell in nb.cells:
        extracted_output = extract_specific_output(cell)
        if extracted_output is not None:
            return extracted_output
    return None

def run_notebooks_in_parallel(input_folder, output_folder, notebook_files):
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    extracted_outputs = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_notebook, input_folder, output_folder, file_name) for file_name in notebook_files]
        for future in concurrent.futures.as_completed(futures):
            output = future.result()
            if output is not None:
                extracted_outputs.append(output)

    return extracted_outputs

# Specify the folders
input_folder = 'Generated_Notebooks'
output_folder = 'Executed_Notebooks0-20'

# Example usage
notebook_files = [f'DropGNP_seed_{i}.ipynb' for i in range(20)]
extracted_outputs = run_notebooks_in_parallel(input_folder, output_folder, notebook_files)

# Write extracted outputs to a file
with open('extracted_outputs.txt', 'w') as f:
    for output in extracted_outputs:
        f.write(f'{output}\n')


In [22]:
#extract value
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
import concurrent.futures
import os


def extract_specific_output(cell):
    uu_value = None
    fitness_value = None
    if cell.cell_type == 'code':
        for output in cell.outputs:
            if output.output_type == 'stream' and 'name' in output and output.name == 'stdout':
                lines = output.text.split('\n')
                for line in lines:
                    if line.strip().isdigit():  # Check if the line is a number
                        if uu_value is None:
                            uu_value = line.strip()
                        else:
                            fitness_value = line.strip()
    return uu_value, fitness_value



def process_notebook(input_folder, output_folder, file_name, index):
    file_path = os.path.join(input_folder, file_name)
    with open(file_path) as f:
        nb = nbformat.read(f, as_version=4)

    ep = ExecutePreprocessor(timeout=30, kernel_name='python3')
    ep.preprocess(nb)

    output_file_path = os.path.join(output_folder, file_name.replace('.ipynb', '_executed.ipynb'))
    with open(output_file_path, 'w', encoding='utf-8') as f:
        nbformat.write(nb, f)

    for cell in nb.cells:
        yy, fitness = extract_specific_output(cell)
        if yy is not None or fitness is not None:
            return index, yy, fitness
    return index, None, None

def run_notebooks_in_parallel(input_folder, output_folder, notebook_files):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    extracted_outputs = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = {executor.submit(process_notebook, input_folder, output_folder, file_name, i): i for i, file_name in enumerate(notebook_files)}
        for future in concurrent.futures.as_completed(futures):
            output = future.result()
            if output is not None:
                extracted_outputs.append(output)

    return extracted_outputs

# Example usage
input_folder = 'Generated_Notebooks200-220'
output_folder = 'Executed_Notebooks200-220'
notebook_files = [f'PickGNP_seed_{i}.ipynb' for i in range(200,220)]
extracted_outputs = run_notebooks_in_parallel(input_folder, output_folder, notebook_files)

# Write extracted outputs to a file
with open('extracted_outputs200-220.txt', 'w') as f:
    for index, yy, fitness in extracted_outputs:
        f.write(f'Index: {index}, yy: {yy}, fitness[0]: {fitness}\n')


In [44]:
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
import concurrent.futures
import os
import re

def extract_specific_output(cell):
    uu_value = None
    fitness_value = None
    if cell.cell_type == 'code':
        for output in cell.outputs:
            if output.output_type == 'stream' and 'name' in output and output.name == 'stdout':
                lines = output.text.split('\n')
                for line in lines:
                    if line.strip().isdigit():  # Check if the line is a number
                        if uu_value is None:
                            uu_value = line.strip()
                        else:
                            fitness_value = line.strip()
    return uu_value, fitness_value

def process_notebook(input_folder, output_folder, file_name):
    # Extract seed index from the filename
    seed_index = int(re.search(r'PickGNP_seed_(\d+).ipynb', file_name).group(1))

    file_path = os.path.join(input_folder, file_name)
    with open(file_path) as f:
        nb = nbformat.read(f, as_version=4)

    ep = ExecutePreprocessor(timeout=30, kernel_name='python3')
    ep.preprocess(nb)

    output_file_path = os.path.join(output_folder, file_name.replace('.ipynb', '_executed.ipynb'))
    with open(output_file_path, 'w', encoding='utf-8') as f:
        nbformat.write(nb, f)

    for cell in nb.cells:
        uu, fitness = extract_specific_output(cell)
        if uu is not None or fitness is not None:
            return seed_index, uu, fitness
    return seed_index, None, None

def run_notebooks_in_parallel(input_folder, output_folder, notebook_files):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    extracted_outputs = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = {executor.submit(process_notebook, input_folder, output_folder, file_name): file_name for file_name in notebook_files}
        for future in concurrent.futures.as_completed(futures):
            output = future.result()
            if output is not None:
                extracted_outputs.append(output)

    return extracted_outputs

# Example usage
input_folder = 'Generated_Notebooks180-200'
output_folder = 'Executed_Notebooks180-200'
notebook_files = [f'PickGNP_seed_{i}.ipynb' for i in range(180,200)]
extracted_outputs = run_notebooks_in_parallel(input_folder, output_folder, notebook_files)

# Write extracted outputs to a file
with open('extracted_outputs180-200.txt', 'w') as f:
    for index, uu, fitness in extracted_outputs:
        f.write(f'Index: {index}, uu: {uu}, fitness[0]: {fitness}\n')


In [3]:
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
import concurrent.futures
import os
import re

def extract_specific_output(cell):
    uu_value = None
    fitness_value = None
    if cell.cell_type == 'code':
        for output in cell.outputs:
            if output.output_type == 'stream' and 'name' in output and output.name == 'stdout':
                lines = output.text.split('\n')
                for line in lines:
                    if line.strip().isdigit():  # Check if the line is a number
                        if uu_value is None:
                            uu_value = line.strip()
                        else:
                            fitness_value = line.strip()
    return uu_value, fitness_value

def process_notebook(input_folder, output_folder, file_name):
    # Extract seed index from the filename
    seed_index = int(re.search(r'DropGNP_seed_(\d+).ipynb', file_name).group(1))

    file_path = os.path.join(input_folder, file_name)
    with open(file_path) as f:
        nb = nbformat.read(f, as_version=4)

    ep = ExecutePreprocessor(timeout=30, kernel_name='python3')
    ep.preprocess(nb)

    output_file_path = os.path.join(output_folder, file_name.replace('.ipynb', '_executed.ipynb'))
    with open(output_file_path, 'w', encoding='utf-8') as f:
        nbformat.write(nb, f)

    for cell in nb.cells:
        yy, fitness = extract_specific_output(cell)
        if yy is not None or fitness is not None:
            return seed_index, yy, fitness
    return seed_index, None, None

def run_notebooks_in_parallel(input_folder, output_folder, notebook_files):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    extracted_outputs = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = {executor.submit(process_notebook, input_folder, output_folder, file_name): file_name for file_name in notebook_files}
        for future in concurrent.futures.as_completed(futures):
            output = future.result()
            if output is not None:
                extracted_outputs.append(output)

    return extracted_outputs

# Example usage
input_folder = 'Generated_Notebooks40-60'
output_folder = 'Executed_Notebooks40-60'
notebook_files = [f'DropGNP_seed_{i}.ipynb' for i in range(40,60)]
extracted_outputs = run_notebooks_in_parallel(input_folder, output_folder, notebook_files)

# Write extracted outputs to a file
with open('Drop_outputs40-60.txt', 'w') as f:
    for index, yy, fitness in extracted_outputs:
        f.write(f'Index: {index}, yy: {yy}, fitness[0]: {fitness}\n')


FileNotFoundError: [Errno 2] No such file or directory: 'Generated_Notebooks40-60\\DropGNP_seed_40.ipynb'

In [1]:
import nbformat as nbf
import os

# Template code as a string
template_code = """
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
import concurrent.futures
import os
import re

def extract_specific_output(cell):
    yy_value = None
    fitness_value = None
    if cell.cell_type == 'code':
        for output in cell.outputs:
            if output.output_type == 'stream' and 'name' in output and output.name == 'stdout':
                lines = output.text.split('\\n')
                for line in lines:
                    if line.strip().isdigit():  # Check if the line is a number
                        if yy_value is None:
                            yy_value = line.strip()
                        else:
                            fitness_value = line.strip()
    return yy_value, fitness_value

def process_notebook(input_folder, output_folder, file_name):
    # Extract seed index from the filename
    seed_index = int(re.search(r'DropGNP_seed_(\\d+).ipynb', file_name).group(1))

    file_path = os.path.join(input_folder, file_name)
    with open(file_path) as f:
        nb = nbformat.read(f, as_version=4)

    ep = ExecutePreprocessor(timeout=30, kernel_name='python3')
    ep.preprocess(nb)

    output_file_path = os.path.join(output_folder, file_name.replace('.ipynb', '_executed.ipynb'))
    with open(output_file_path, 'w', encoding='utf-8') as f:
        nbformat.write(nb, f)

    for cell in nb.cells:
        yy, fitness = extract_specific_output(cell)
        if yy is not None or fitness is not None:
            return seed_index, yy, fitness
    return seed_index, None, None

def run_notebooks_in_parallel(input_folder, output_folder, notebook_files):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    extracted_outputs = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = {executor.submit(process_notebook, input_folder, output_folder, file_name): file_name for file_name in notebook_files}
        for future in concurrent.futures.as_completed(futures):
            output = future.result()
            if output is not None:
                extracted_outputs.append(output)

    return extracted_outputs

# Example usage
input_folder = 'GenD0-20'
output_folder = 'Executed_Notebooks40-60'
notebook_files = [f'DropGNP_seed_{i}.ipynb' for i in range(40,60)]
extracted_outputs = run_notebooks_in_parallel(input_folder, output_folder, notebook_files)

# Write extracted outputs to a file
with open('Drop_outputs40-60.txt', 'w') as f:
    for index, yy, fitness in extracted_outputs:
        f.write(f'Index: {index}, yy: {yy}, fitness[0]: {fitness}\\n')
"""

# Generating 20 different .ipynb files with modified code
for i in range(0, 20):
    # Modify the code
    modified_code = template_code.replace("input_folder = 'Generated_Notebooks40-60'", f"input_folder = 'Generated_Notebooks{i*20}-{(i+1)*20}'")
    modified_code = modified_code.replace("output_folder = 'Executed_Notebooks40-60'", f"output_folder = 'Executed_Notebooks{i*20}-{(i+1)*20}'")
    modified_code = modified_code.replace("notebook_files = [f'DropGNP_seed_{i}.ipynb' for i in range(40,60)]", f"notebook_files = [f'DropGNP_seed_{{j}}.ipynb' for j in range({i*20}, {(i+1)*20})]")

    # Create a new notebook
    nb = nbf.v4.new_notebook()
    code_cell = nbf.v4.new_code_cell(modified_code)
    nb.cells.append(code_cell)

    # Write the modified content to a new notebook file
    output_file_name = f'modified_script_{i*20}-{(i+1)*20}.ipynb'
    with open(output_file_name, 'w') as new_file:
        nbf.write(nb, new_file)


In [1]:
#drop executor generator

import nbformat as nbf
import os

# Template code as a string
template_code = """
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
import concurrent.futures
import os
import re

def extract_specific_output(cell):
    yy_value = None
    fitness_value = None
    if cell.cell_type == 'code':
        for output in cell.outputs:
            if output.output_type == 'stream' and 'name' in output and output.name == 'stdout':
                lines = output.text.split('\\n')
                for line in lines:
                    if line.strip().isdigit():  # Check if the line is a number
                        if yy_value is None:
                            yy_value = line.strip()
                        else:
                            fitness_value = line.strip()
    return yy_value, fitness_value

def process_notebook(input_folder, output_folder, file_name):
    # Extract seed index from the filename
    seed_index = int(re.search(r'DropGNP_seed_(\\d+).ipynb', file_name).group(1))

    file_path = os.path.join(input_folder, file_name)
    with open(file_path) as f:
        nb = nbformat.read(f, as_version=4)

    ep = ExecutePreprocessor(timeout=30, kernel_name='python3')
    ep.preprocess(nb)

    output_file_path = os.path.join(output_folder, file_name.replace('.ipynb', '_executed.ipynb'))
    with open(output_file_path, 'w', encoding='utf-8') as f:
        nbformat.write(nb, f)

    for cell in nb.cells:
        yy, fitness = extract_specific_output(cell)
        if yy is not None or fitness is not None:
            return seed_index, yy, fitness
    return seed_index, None, None

def run_notebooks_in_parallel(input_folder, output_folder, notebook_files):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    extracted_outputs = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = {executor.submit(process_notebook, input_folder, output_folder, file_name): file_name for file_name in notebook_files}
        for future in concurrent.futures.as_completed(futures):
            output = future.result()
            if output is not None:
                extracted_outputs.append(output)

    return extracted_outputs

# Example usage
input_folder = 'GenD0-20'
output_folder = 'ExecD0-20'
notebook_files = [f'DropGNP_seed_{i}.ipynb' for i in range(0, 20)]
extracted_outputs = run_notebooks_in_parallel(input_folder, output_folder, notebook_files)

# Write extracted outputs to a file
with open('Drop_outputs0-20.txt', 'w') as f:
    for index, yy, fitness in extracted_outputs:
        f.write(f'Index: {index}, yy: {yy}, fitness[0]: {fitness}\\n')
"""

# Generating 20 different .ipynb files with modified code
for i in range(0, 20):
    # Modify the code
    range_start = i * 20
    range_end = (i + 1) * 20
    modified_code = template_code.replace("input_folder = 'GenD0-20'", f"input_folder = 'GenD{range_start}-{range_end}'")
    modified_code = modified_code.replace("output_folder = 'ExecD0-20'", f"output_folder = 'ExecD{range_start}-{range_end}'")
    modified_code = modified_code.replace("notebook_files = [f'DropGNP_seed_{i}.ipynb' for i in range(0, 20)]", f"notebook_files = [f'DropGNP_seed_{{j}}.ipynb' for j in range({range_start}, {range_end})]")
    modified_code = modified_code.replace("'Drop_outputs0-20.txt'", f"'Drop_outputs{range_start}-{range_end}.txt'")

    # Create a new notebook
    nb = nbf.v4.new_notebook()
    code_cell = nbf.v4.new_code_cell(modified_code)
    nb.cells.append(code_cell)

    # Write the modified content to a new notebook file
    output_file_name = f'modified_script_{range_start}-{range_end}.ipynb'
    with open(output_file_name, 'w') as new_file:
        nbf.write(nb, new_file)
